Url: https://tbrain.trendmicro.com.tw/Competitions/Details/2

In [2]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window

In [3]:
global Path
if sc.master[0:5]=="local":
    Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180427"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180427"
else:
    Path = "hdfs://master:9000/user/hduser"

In [ ]:
sc.master

In [ ]:
float("    46.57")+float("     29,020".replace(",",""))

In [4]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}

In [5]:
#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [6]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [7]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 6515
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50      

In [8]:
#EDA
#range of date
tetfp_dt.describe('etf_date').show()

+-------+--------------------+
|summary|            etf_date|
+-------+--------------------+
|  count|                6515|
|   mean|2.0152356018419035E7|
| stddev|  15378.548560182882|
|    min|            20130102|
|    max|            20180427|
+-------+--------------------+



In [ ]:
print("starting import taetfp.csv(台灣18檔ETF調整後股價資料)...")
taetfp_dt=read_data("taetfp.csv", col_alias_etf)

In [ ]:
print("starting import tsharep.csv(台灣個股股價資料)...")
tsharep_dt=read_data("tsharep.csv", col_alias_stock)

In [ ]:
print("starting import tasharep.csv(台灣個股調整後股價資料)...")
tasharep_dt=read_data("tasharep.csv", col_alias_stock)

In [9]:
import sys
from pyspark.sql.functions import lag, col, avg,collect_list, lit
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, DoubleType
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
wsSpec_etf_close_price_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, -1)
wsSpec_etf_dif_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, 0)
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)
#計算EMA的udf
def calculate_ema_native(close_p_list, window_len):
    #透過歷史收盤價計算
    if len(close_p_list) < window_len:
        return None
    elif len(close_p_list) == window_len:
        #if len of list = win_len then return avg, 
        return avg_list(close_p_list)
    else:
        #else EMA[t] =(EMA[t-1]*(win_len-1)+close[t]*2)/(win_len+1)
        ema = avg_list(close_p_list[:window_len])
        for price in close_p_list[window_len:]:
            ema = (ema*(window_len-1)+price*2)/(window_len+1)
        return ema
calculate_ema=udf(calculate_ema_native, DoubleType())
#計算BIAS的udf
def calculate_bias(close_p_list):
    #計算前日收盤價與N日均線之差比: (close price - MA)/MA   ,Paper 建議用20日MA
    #因要預測今日的收盤價，故計算前日收盤價與前20日均線
    if len(close_p_list) < 21:
        return None
    else:
        list_len = len(close_p_list)
        p_close = close_p_list[-1]
        cal_list = close_p_list[list_len-21: list_len-1]
        return p_close - avg_list(cal_list)
calculate_bias=udf(calculate_bias, DoubleType())

def get_min_max_last(p_list):
    #找出list中最大最小和最後一個值, 回傳(min, max, last)
    return (min(p_list), max(p_list), p_list[-1])
def calculate_raw_rsv(p_list):
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    p_min, p_max, p_last = get_min_max_last(p_list)
    rsv = (p_last - p_min)/(p_max - p_min)
    return rsv
def calculate_rsv(p_9_list, k_prev, d_prev):
    #計算加權後的RSV，p_9_list=>9日收盤價
    rrsv = calculate_raw_rsv(p_9_list)
    k_curr = (1/3)*rrsv + (2/3)*k_prev
    d_curr = (1/3)*k_curr + (2/3)*d_prev
    return [k_curr, d_curr]
#計算隨機指標（Stochastic Oscillator，KD），原名%K&%D
def calculate_KD(close_p_list):
    win_len = 9 #看過去 9 日值
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    #K_curr = 1/3*RSV + 2/3*K_prev
    #D_curr = 1/3*K_curr + 2/3*D_prev
    if len(close_p_list) < win_len:
        return None
    elif len(close_p_list) == win_len:
        #無前日K, D時，以0.5帶入
        return calculate_rsv(close_p_list, 0.5, 0.5)
    else:
        kds = calculate_rsv(close_p_list[0:9], 0.5, 0.5)
        for idx in range(1, (len(close_p_list)+1-9)):
            p_9_list = close_p_list[idx: idx+9]
            kds = calculate_rsv(p_9_list, kds[0], kds[1])
        return kds
calculate_KD=udf(calculate_KD, ArrayType(DoubleType()))

#計算差離值DIF = 12日EMA - 26日EMA
def calculate_DIF(close_p_list):
    if len(close_p_list) < 26:
        return None
    else:
        ema12 = calculate_ema_native(close_p_list, 12)
        ema26 = calculate_ema_native(close_p_list, 26)
        return ema12 - ema26
calculate_DIF=udf(calculate_DIF, DoubleType())

#計算MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
def calculate_MACD(dif_list, dif_curr):
    win_len = 9
    if len(dif_list) < win_len:
        return None
    elif len(dif_list) == win_len:
        #if len of list = win_len then return avg, 
        return avg_list(dif_list)
    else:
        #MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
        macd = avg_list(dif_list[:win_len])
        for price in dif_list[win_len:]:
            macd = (macd*(win_len-1)+dif_curr*2)/(win_len+1)
        return macd
calculate_MACD=udf(calculate_MACD, DoubleType())

#計算相對強弱指數(RSI)
def calculate_RSI(close_p_list):
    win_len = 9
    if len(close_p_list) < (win_len + 1):
        return None
    else:
        cur_list = close_p_list[1:]
        prv_list = close_p_list[0:-1]
        p_dif_list = list(map(lambda x,y : x - y, cur_list, prv_list)) #dif list
        u_list = []
        d_list = []
        for dif in p_dif_list:
            if dif == 0:
                #若兩天價格相同，則U及D皆等於零
                u_list.append(0)
                d_list.append(0)
            elif dif > 0:
                #在價格上升的日子, U = diff, D = 0
                u_list.append(dif)
                d_list.append(0)
            else:
                #在價格下跌的日子, U = 0, D = abs(diff)
                u_list.append(0)
                d_list.append(abs(dif))
        #RSI = ema(u,9)/(ema(u,9)+ema(d,9))
        ema_u = calculate_ema_native(u_list, win_len)
        ema_d = calculate_ema_native(d_list, win_len)
        return ema_u/(ema_u + ema_d)
calculate_RSI=udf(calculate_RSI, DoubleType())

#計算威廉指標（Williams %R）
def calculate_WR(close_p_list):
    win_len = 9
    if len(close_p_list) < win_len:
        return None
    else:
        p_list = close_p_list[len(close_p_list) - win_len :]
        return 1.0 - calculate_raw_rsv(p_list)
calculate_WR=udf(calculate_WR, DoubleType())


In [10]:
#calculate ema [5,10,20] #cannot remove row_idx, row_idx for next window usage
tetfp_dt2=tetfp_dt.withColumn("row_idx", rank().over(wsSpec_etf)) \
    .withColumn("close_price_raw", collect_list(col('etf_close')).over(wsSpec_etf_close_price_raw)) \
    .withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("DIF", calculate_DIF(col("close_price_raw"))) \
    .withColumn("dif_list", collect_list(col('DIF')).over(wsSpec_etf_dif_raw)) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))

tetfp_dt2.cache()
tetfp_dt2.printSchema()

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)
 |-- row_idx: integer (nullable = true)
 |-- close_price_raw: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- EMA5: double (nullable = true)
 |-- EMA10: double (nullable = true)
 |-- EMA20: double (nullable = true)
 |-- BIAS: double (nullable = true)
 |-- KD: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- K: double (nullable = true)
 |-- D: double (nullable = true)
 |-- DIF: double (nullable = true)
 |-- dif_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- MACD: double (nullable = true)
 |-- RSI: double (nullable = true)
 |-- WR: double (nullable = true)



In [ ]:
tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_close", "MACD", "RSI", "WR") \
            .show(45)
#          .toPandas().to_csv(Path.replace("file:","") + "/taetfp_BIAS.csv",index=False)

In [ ]:
#匯出成pandas
etf_pd_50 = tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_date" ,"etf_close", "EMA5", "EMA10", "EMA20", "BIAS") \
                .toPandas()

In [ ]:
#圖形化顯示
#aetf_pd_50.set_index('etf_date') #set_index 後配合 loc select 出區段資料來看
etf_pd_50_part = etf_pd_50.loc[1200:]
etf_pd_50_part.etf_close.plot(x='row_idx', y='etf_close', style='b--', label="etf_close")
etf_pd_50_part.EMA5.plot(x='row_idx', y='EMA5', label="EMA5", style='r-')
etf_pd_50_part.EMA10.plot(x='row_idx', y='EMA10', label="EMA10", style='g-')
etf_pd_50_part.EMA20.plot(x='row_idx', y='EMA20', label="EMA20", style='y-')
plt.legend()
plt.show()

In [ ]:
#計算各欄位與收盤價之相關性
corr_cols = ['EMA5','EMA10','EMA20','BIAS','K','D']
for col in corr_cols:
    print('corr between ', col , ' and etf_close: ', str(tetfp_dt2.corr(col, 'etf_close')))

In [ ]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors

dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(dataFrame)

# rescale each feature to range [min, max].
scaledData = scalerModel.transform(dataFrame)
#print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
scaledData.select("features", "scaledFeatures").show()

In [12]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors
tot_dt = tetfp_dt2.filter("MACD is not null") \
    .select("etf_id", "etf_date", "EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
    
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#將Feature合併為Vector 並作標準化
assembler = VectorAssembler(
    inputCols=["EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR"],
    outputCol="features")
tot_dt_1 = assembler.transform(tot_dt)
#minmax_scaler = MinMaxScaler(inputCol="features", outputCol="stdFeatures")
std_scaler = StandardScaler(inputCol="features", outputCol="stdFeatures")
scaler_model = std_scaler.fit(tot_dt_1)
tot_dt_scale = scaler_model.transform(tot_dt_1)


In [13]:
#取出4/16~4/27 (共兩週資料作為測試集)
train_dt = tot_dt_scale.filter("etf_date < '20180416' and MACD is not null") \
    .select("etf_id", "etf_date", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
test_dt = tot_dt_scale.filter("etf_date >= '20180416'") \
    .select("etf_id", "etf_date", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
print('train count: ', str(train_dt.count()), ', test count: ', str(test_dt.count()))
train_dt.show(20)
test_dt.show(20)

train count:  6295 , test count:  50
+-------+--------+--------------------+---------+
| etf_id|etf_date|         stdFeatures|etf_close|
+-------+--------+--------------------+---------+
|0050   |20130227|[3.256641619336,3...|     55.2|
|0050   |20130301|[3.25111747966177...|     55.4|
|0050   |20130304|[3.25134784693247...|    54.75|
|0050   |20130305|[3.23878376218901...|     55.2|
|0050   |20130306|[3.23921224156378...|    55.45|
|0050   |20130307|[3.24438930329719...|     55.4|
|0050   |20130308|[3.24686239602275...|     55.8|
|0050   |20130311|[3.25633737861350...|     55.9|
|0050   |20130312|[3.26461059720076...|    55.55|
|0050   |20130313|[3.26327810391527...|    55.65|
|0050   |20130314|[3.26434633858504...|     55.6|
|0050   |20130315|[3.26408021326817...|     55.2|
|0050   |20130318|[3.25607654228322...|     54.5|
|0050   |20130319|[3.23704481693927...|     54.6|
|0050   |20130320|[3.22631356357007...|    54.25|
|0050   |20130321|[3.21231142231361...|     54.3|
|0050   |2013

In [14]:
#取出etf的distinct id
etf_ids = []
for row in test_dt.select("etf_id").distinct().collect():
    etf_ids.append(row["etf_id"])
etf_ids

['0051   ', '0052   ', '0050   ', '0054   ', '0053   ']

In [15]:
#訓練Model及評估
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
rf = RandomForestRegressor(featuresCol="stdFeatures",labelCol="etf_close")
predit_res = None
for etfid in etf_ids:
    train_data = train_dt.filter("etf_id='" + etfid + "'")
    test_data = test_dt.filter("etf_id='" + etfid + "'")
    rf_model = rf.fit(train_data)
    predicts = rf_model.transform(test_data)
    if predit_res is None:
        predit_res = predicts
    else:
        predit_res = predit_res.unionAll(predicts)
predit_res.show(10)
#評估RMES
evaluator = RegressionEvaluator(
    labelCol="etf_close", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predit_res)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


+-------+--------+--------------------+---------+------------------+
| etf_id|etf_date|         stdFeatures|etf_close|        prediction|
+-------+--------+--------------------+---------+------------------+
|0051   |20180416|[1.92327568946285...|    33.18|  32.2298144070044|
|0051   |20180417|[1.93137157115395...|     33.0|32.271958337750206|
|0051   |20180418|[1.93324701126652...|    32.75|32.251593526553286|
|0051   |20180419|[1.92960589585800...|    33.15| 32.20944959580747|
|0051   |20180420|[1.93500473969269...|    33.02|32.271958337750206|
|0051   |20180423|[1.93606043633103...|     33.0|32.251593526553286|
|0051   |20180424|[1.93637292138458...|    32.44|32.251593526553286|
|0051   |20180425|[1.92562448900375...|    32.02| 32.03906841425838|
|0051   |20180426|[1.91024130060415...|    31.71|  31.5904495488314|
|0051   |20180427|[1.89392049473812...|    32.11|31.412555655872506|
+-------+--------+--------------------+---------+------------------+
only showing top 10 rows

Root Mea

In [17]:
#計算上或下的值
def judge_up_down(curr_price, prev_price):
    if prev_price is None:
        return 0
    elif curr_price == prev_price:
        return 0
    elif curr_price > prev_price:
        return 1
    else:
        return -1
judge_up_down=udf(judge_up_down, DoubleType())
predit_res_2 = predit_res.withColumn("prev_close", lag("etf_close").over(wsSpec_etf)) \
    .withColumn("prev_pred_close", lag("prediction").over(wsSpec_etf)) \
    .withColumn("act_ud", judge_up_down(judge_up_down(col("etf_close"), col("prev_close")))) \
    .withColumn("pred_ud", judge_up_down(judge_up_down(col("prediction"), col("prev_pred_close"))))
predit_res_2.printSchema()
predit_res_2.show()

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- stdFeatures: vector (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- prediction: double (nullable = true)
 |-- prev_close: double (nullable = true)

+-------+--------+--------------------+---------+------------------+----------+
| etf_id|etf_date|         stdFeatures|etf_close|        prediction|prev_close|
+-------+--------+--------------------+---------+------------------+----------+
|0051   |20180416|[1.92327568946285...|    33.18|  32.2298144070044|      null|
|0051   |20180417|[1.93137157115395...|     33.0|32.271958337750206|     33.18|
|0051   |20180418|[1.93324701126652...|    32.75|32.251593526553286|      33.0|
|0051   |20180419|[1.92960589585800...|    33.15| 32.20944959580747|     32.75|
|0051   |20180420|[1.93500473969269...|    33.02|32.271958337750206|     33.15|
|0051   |20180423|[1.93606043633103...|     33.0|32.251593526553286|     33.02|
|0051   |20180424|[1.9363729

In [ ]:
#ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66, 47.0, 46.96, 47.0]
ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66]
win_len=5
print(ll[1:])
print(ll[0: -1])
print(list(map(lambda x,y : x - y, ll[1:], ll[0: -1])))
for x in ll[win_len:]:
    print(x)
ema = sum(ll[:win_len])/len(ll[:win_len])
print(ema)
for price in ll[win_len:]:
    ema = (ema*(win_len-1)+price*2)/(win_len+1)
tup1, tup2 = (1,2)
print(tup1, ' ', tup2)
tup = (3,4)
print(tup[0], ' ', tup[1])
list(range(0,2))